# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [21]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [22]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [23]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [24]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [25]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [26]:
#Code below creates a keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """
    
    )
except Exception as e:
    print(e)


#### Set Keyspace and Write query functions

In [27]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)
    
def execute_query(query):
    """
    This function execute query
    Input:
    :param query: the query
    """
    try:
        return session.execute(query)
    except Exception as e:
        print(e)

In [28]:
#Table structure - Identifier - session_id, item_in_session. Info - artist, song, song_length
execute_query("""
    CREATE TABLE IF NOT EXISTS session_library(
       session_id int, 
       item_in_session int, 
       artist text,
       song text, 
       song_length decimal, 
       PRIMARY KEY (session_id, item_in_session)
    )
"""
)

                    

In [29]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO session_library (session_id, item_in_session, artist, song, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [30]:
#Verifying that the data has been inserted into each table
#By running a query that answers session_id = 338, item_in_session = 4
#The above return artist, song, length from the session_library

query = """SELECT artist, song, song_length FROM session_library WHERE session_id = 338 AND item_in_session = 4"""

try:
    songs = session.execute(query)
except Exception as e:
    print (e)

table = PrettyTable(['Artist', 'Song', 'Song Length'])
for row in songs:
    table.add_row([row.artist, row.song, row.song_length])
print(table)

+-----------+---------------------------------+-------------+
|   Artist  |               Song              | Song Length |
+-----------+---------------------------------+-------------+
| Faithless | Music Matters (Mark Knight Dub) |   495.3073  |
+-----------+---------------------------------+-------------+


## Query 2 section

In [31]:
#Table Structure - Identifier - user-id, session_id, ORDER -item_in_session, Information - artist, song, first_name, last_name

execute_query("""
  CREATE TABLE IF NOT EXISTS artist_details (
    user_id int,
    session_id int, 
    item_in_session int,
    artist text,
    song text,
    first_name text,
    last_name text, 
    PRIMARY KEY ((user_id, session_id), item_in_session)
  
  )
"""
)


                    

In [32]:
#At this point Insert Data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO artist_details (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [33]:
## Query name of artist, song(sorted by item_in_session) Userid = 10, sessionid = 182
query = """SELECT item_in_session, artist, song, first_name, last_name FROM artist_details WHERE user_id = 10 AND session_id = 182    
"""
try:
    artists = session.execute(query)
except Exception as e:
    print (e)

table2 = PrettyTable(['Item in Session', 'Artist', 'Song', 'First Name', 'Last Name' ])

for row in artists:
    table2.add_row([row.item_in_session, row.artist, row.song, row.first_name, row.last_name])
print(table2)

+-----------------+-------------------+------------------------------------------------------+------------+-----------+
| Item in Session |       Artist      |                         Song                         | First Name | Last Name |
+-----------------+-------------------+------------------------------------------------------+------------+-----------+
|        0        |  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|        1        |    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
|        2        | Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|        3        |   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-----------------+-------------------+------------------------------------------------------+------------+-----------+


## Query 3 Section  

In [34]:
execute_query("""
 CREATE TABLE IF NOT EXISTS user_details(
 user_id int, 
 song text, 
 artist text,
 first_name text, 
 last_name text,
 PRIMARY KEY (song, user_id)
 )  
"""
)

In [35]:
# Insert Data for Query 3
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT into user_details (user_id, song, artist, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[9], line[0], line[1], line[4]))

## Query Test
Query for first_name, last_name, song, artist from the table. Given Song is 'All Hands Against His Own'

In [36]:
query = """ SELECT first_name, last_name, song, artist FROM user_details WHERE song = 'All Hands Against His Own'"""

try:
    users = session.execute(query)
except Exception as e:
    print (e)

table3 = PrettyTable(['First Name', 'Last Name', 'Song', 'Artist'])

for row in users:
    table3.add_row([row.first_name, row.last_name, row.song, row.artist])
print(table3)

+------------+-----------+---------------------------+----------------+
| First Name | Last Name |            Song           |     Artist     |
+------------+-----------+---------------------------+----------------+
| Jacqueline |   Lynch   | All Hands Against His Own | The Black Keys |
|   Tegan    |   Levine  | All Hands Against His Own | The Black Keys |
|    Sara    |  Johnson  | All Hands Against His Own | The Black Keys |
+------------+-----------+---------------------------+----------------+


### Drop the tables before closing out the sessions

In [37]:
execute_query("DROP TABLE session_library")
execute_query("DROP TABLE artist_details")
execute_query("DROP TABLE user_details")

### Close the session and cluster connection¶

In [38]:
session.shutdown()
cluster.shutdown()